## ZTF alerts demo

In this demo, we will:
- Set up `Docker`
- Using `Docker`, deploy a `MongoDB` database, fetch a night worth of (pre-filtered to reduce size) public ZTF alerts, ingest it into the database, and create indices in the database for faster queries
- Set up `Robo3T` and use it to look at the database and query it
- Query the database using `python`
- Inspect the contents of an alert packet
- Construct and plot a light curve
- Plot the cutout images from an alert packet


### Download and install `Docker` and `Robo3T`
- Download and install the appropriate version of `Docker` for your platform from [here](https://www.docker.com/community-edition). You will need to create an account on their website.
- Download and install `Robo3T` from [here](https://robomongo.org/download).
  We will use it to connect to the database.

### Fetch, build, and run the code to deploy a `MongoDB` database, fetch a night worth of (pre-filtered to reduce size) public ZTF alerts, ingest it into the database, and create indices in the database for faster queries

This is a lot of stuff! Sounds scary, however with the help of `Docker`, we will only have to run a few simple commands to do all that.

Clone the repo and `cd` into the directory:

```bash
git clone https://github.com/dmitryduev/ztf-alerts-demo.git
cd ztf-alerts-demo
```

The inside/contents of a `Docker` container get destroyed when it is removed, so we need to tell `Docker` to keep the useful data in a "persistent" storage.

Create a so-called persistent `Docker` volume for `MongoDB`:
```bash
docker volume create alert-fetcher-mongo-volume
```

Launch the `MongoDB` container. (Feel free to change u/p for the db admin)
```bash
docker run -d --restart always --name alert-fetcher-mongo -p 27018:27017 \
       -v alert-fetcher-mongo-volume:/data/db \
       -e MONGO_INITDB_ROOT_USERNAME=mongoadmin \
       -e MONGO_INITDB_ROOT_PASSWORD=mongoadminsecret \
       mongo:latest
```

That's it for the database set-up! You can now connect to it on `localhost` on port `27018`.

Finally, build and launch the alert-fetcher container. We will bind-mount a directory on your host machine to store the alerts:
```bash
cd alert-fetcher
docker build -t alert-fetcher -f Dockerfile .
# make sure path ./alerts (or whatever path you specify) exists
docker run -v ./alerts:/alerts \
           --name alert-fetcher -d --link alert-fetcher-mongo:mongo -it alert-fetcher
```

This will launch a program that will fetch a pre-filtered (demo) set of public ZTF alerts from July 13, 2018, ingest that into the MongoDB database, and create indicies to accelerate queries. The script will add a `coordinates` field to each that is not part of the original alert packets, to allow 2d indexation on the sphere for fast positional/cone searches. Additionally, a unique string `candid_objectId` is used a the alert identifier in the database.

---
We can now use `Robo3T` to connect to the database and take a look at the alerts. Launch `Robo3T` and click "create" connection. 
- Name the connection, for example, `local_docker_alerts_demo`
- Use `localhost` : `27018` as the address
- On the `Authentication` tab, check "perform authentication", and use `admin` for `Database`, `mongoadmin` for `User Name`, `mongoadminsecret` for 'Password'(if you did not choose other u/p), and `SCRAM-SHA-1` for `Auth Mechanism`
- Click `Save` and then `Connect`

`Robo3T` will connect to the database. To see the ingested alerts, click `ztf_alerts` -> `Collections` -> `alerts`.

---
##### Notes [optional]

To check the running/stopped containers, type:
```bash
docker ps -a
```

To stop and remove a container, run:
```bash
docker stop container_name
docker rm -f container_name
```

To get all public alerts from the ZTF archive for a given night, remove `--demo` from the last line of the file `alert-fetcher/Dockerfile` and change the date string. Then re-build and re-run the alert-fetcher container.

Every time when you change the code that is used inside a container, the latter must be re-build and restarted.

Alerts are stored in the database in a serialized binary format that resembles the Apache Avro format used in the packets in that it can be easily converted into `json` or a `python` dictionary.

---

We will now explore the contents of an avro packet. We will also learn how to construct and display a light curve for an alert and plot the cutout images.

_NOTE:_ You may want to look at Erik Bellm's Jupyter notebook with some more examples.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import fastavro

from astropy.time import Time
from astropy.io import fits
import aplpy

%matplotlib inline

Let's look at a particular avro packet:

In [ ]:
avro_packet = './alerts/20180601/516170410015015005.avro'
with open(avro_packet, 'rb') as fa:
    freader = fastavro.reader(fa)
    schema = freader.schema

    for packet in freader:
        print(packet.keys())

The schema is stored in the packet:

In [ ]:
schema

The payload, once in memory, is a python dictionary, so the attributes are easy to access

In [ ]:
packet

In [ ]:
print('JD: {} Filter: {} Mag: {:.2f}+/-{:.2f}'.format(
    packet['candidate']['jd'],packet['candidate']['fid'],
    packet['candidate']['magpsf'],packet['candidate']['sigmapsf']))

NOTE ESPECIALLY: the magnitudes here do not include the magnitude of the underlying reference source (if present), so if this is a variable star further adjustment is needed. Example to come...

Record access like this is a little verbose; let's wrap things up in a dataframe for ease of access (and faster loading).

Now let's extract the lightcurves. The alert packet formats are nested, so the historical detections (if present) have the same structure as the candidate triggering the alert (minus a couple fields).

In [ ]:
def make_dataframe(packet):
    df = pd.DataFrame(packet['candidate'], index=[0])
    df_prv = pd.DataFrame(packet['prv_candidates'])
    return pd.concat([df, df_prv], ignore_index=True)

In [ ]:
dflc = make_dataframe(packet)
dflc

We see that some of the historical detections are upper limits, signified by the NaNs. Note that the most recent candidate has a few fields that are not present for the prv_candidates.

Let's plot it!

In [ ]:
def plot_lightcurve(dflc, days_ago=True):
    
    filter_color = {1: 'green', 2: 'red', 3: 'pink'}
    if days_ago:
        now = Time.now().jd
        t = dflc.jd - now
        xlabel = 'Days Ago'
    else:
        t = dflc.jd
        xlabel = 'Time (JD)'
    
    plt.figure()
    for fid, color in filter_color.items():
        # plot detections in this filter:
        w = (dflc.fid == fid) & ~dflc.magpsf.isnull()
        if np.sum(w):
            plt.errorbar(t[w], dflc.loc[w, 'magpsf'], dflc.loc[w, 'sigmapsf'],
                         fmt='.', color=color)
        wnodet = (dflc.fid == fid) & dflc.magpsf.isnull()
        if np.sum(wnodet):
            plt.scatter(t[wnodet], dflc.loc[wnodet, 'diffmaglim'], 
                        marker='v', color=color, alpha=0.25)
    
    plt.gca().invert_yaxis()
    plt.xlabel(xlabel)
    plt.ylabel('Magnitude')

In [ ]:
plot_lightcurve(dflc)

Now let's figure out how to display the cutout images. These are gzip-compressed fits files stored as bytes